In [1]:
# 一、使用__slots__

正常情况下，当我们定义了一个class，创建了一个class的实例后，可以给该实例绑定任何属性和方法，这就是动态语言的灵活性。

In [2]:
class Std(object):
    pass

In [3]:
# 尝试绑定一个属性

s = Std()

In [4]:
# 动态给实例绑定一个属性

s.name = 'ss'

In [5]:
print(s.name)

ss


In [6]:
# 尝试给实例绑定一个方法

def set_age(self, age): # 定义一个函数作为实例方法
    self.age = age

In [7]:
from types import MethodType

In [8]:
s.set_age = MethodType(set_age, s) # 给实例绑定一个方法

In [9]:
s.set_age(25) # 调用实例方法

In [10]:
s.age

25

In [11]:
# 给一个实例绑定的方法，对另一个实例是不起作用的

s2 = Std() # 创建新的实例

In [12]:
s2.set_age(25) # 尝试调用方法

AttributeError: 'Std' object has no attribute 'set_age'

In [ ]:
# 给所有实例都绑定方法，可以给class绑定方法

def set_score(self, score):
    self.score = score

In [ ]:
Std.set_score = set_score

In [ ]:
# 给class绑定方法后，所有实例均可调用

s.set_score(100)

In [ ]:
s.score

In [ ]:
s2.set_score(99)

In [13]:
s2.score

AttributeError: 'Std' object has no attribute 'score'

通常情况下，上面的set_score方法可以直接定义在class中，但动态绑定允许我们在程序中运行的过程中动态给class加上功能。

In [ ]:
# Python允许在定义class的时候，定义一个特殊的__slots__变量，来限制该class实例能添加的属性

class Std(object):
    __slots__ = ('name', 'age') # 用tuple定义允许绑定的属性名称

In [ ]:
s = Std() # 创建新的实例

In [ ]:
s.name = 'ss' # 绑定属性'name'

In [ ]:
s.age = 25 # 绑定属性'age'

In [ ]:
s.score = 100 # 绑定属性'score'

In [ ]:
# 使用__slots__要注意，__slots__定义的属性仅对当前实例起作用，对继承的子类是不起作用

class GraduateStd(Std):
    pass

In [ ]:
g = GraduateStd()

In [ ]:
g.score = 99

In [ ]:
# 二、 使用@property

在绑定属性时，如果我们直接把属性暴露出去，虽然写起来简单但是无法检查参数，导致可以把成绩随便改

In [ ]:
class Student(object):
    pass

In [ ]:
s = Student()

In [14]:
s.score = 99

为了限制score的范围，可以通过一个set_score()方法来设置成绩，再通过一个get_score()来获取成绩，在set_score（）方法里，就可以检查参数

In [15]:
class Student(object):
    
    def get_score(self):
        return self._score
    
    def set_score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 - 100!')
        self._score = value

In [16]:
# 对任意的Student实例进行操作，就不能随心所欲地设置score

s = Student()

In [17]:
s.set_score(60) # ok!

In [18]:
s.get_score()

60

In [19]:
s.set_score(101)

ValueError: score must between 0 - 100!

In [ ]:
# 上面的方法有点复杂，没有直接用属性这么直接简单
# Python内置的@property装饰器就是负责把一个方法变成属性调用

class Student(object):
    
    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must between 0 - 100!')
        self._score = value

In [ ]:
# @score.setter负责把一个setter方法变成属性赋值，于是，我们就拥有一个可控得属性操作

s = Student()

In [ ]:
s.score = 60 # 实际转化为s.set_score(60)

In [ ]:
s.score # 实际转化为s.get_score()

In [ ]:
s.score = 9999

注意到这个神奇得@property，在对实例属性操作的时候，就知道该属性很可能不是直接暴露的，而是通过getter和setter方法来实现。

In [20]:
# 还可以定义只读属性，只定义getter方法，不定义setter方法就是一个只读属性

class Student(object):
    
    @property
    def birth(self):
        return self._birth
    
    @birth.setter
    def birth(self, value):
        self._birth = value
        
    @property
    def age(self):
        return 2020 - self._birth

birth是可读写属性，而age就是一个只读属性，因为age可以根据birth和当前时间计算出来。

## 小结

@property广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，程序运行时就减少了出错的可能性。

In [21]:
# 三、 多重继承

继承是面向对象编程的一个重要的方式，因为通过继承，子类就可以扩展父类的功能。

设计一个Animal类层次，假设有以下四种动物：

Dog、Bat、Parrot、Ostrich

如果按照哺乳动物和鸟类归类：

Mammal：Dog、Bat
Bird：Parrot、Ostrich

如果按照能跑和能飞归类

Runnable:Dog、Ostrich
Flyable：Parrot、Bat

如果把以上两种分类都包含进来，就可以设计出更多的层次：

哺乳类：能跑的哺乳类、能飞的哺乳类
鸟类：能跑的鸟类、能飞的鸟类

Mammal：MRun、MFly
Bird：BRun、MFly

显然这样设计下去，类的数量会呈指数增长，设计的不好。


In [22]:
# 正确做法采用多重继承，首先主要类层次按照哺乳类和鸟类设计

class Animal(object):
    pass

class Mammal(Animal):
    pass

class Bird(Animal):
    pass

class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

In [23]:
# 现在给动物加上Runnable和Flyable的功能，只需要先定义好这两个类即可

class Runnable(object):
    def run(self):
        print('Running...')
        
class Flyable(object):
    def fly(self):
        print('Flying...')

In [24]:
# 需要Runnable功能的动物，就多继承一个Runnable

class Dog(Mammal, Runnable):
    pass

In [25]:
class Bat(Mammal, Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的所有功能。

In [26]:
## 3.1 MixIn

在设计类的继承关系时，通常，主线都是单一继承下来。例如，Ostrich继承Bird，但是如果混入额外的功能，通过多重继承就可以实现，让Ostrich除了继承Bird外再继承Runnable。这种设计通常称为MixIn。

为了更好看出继承关系，把Runnable和Flyable改为RunnableMixIn和FlyableMixIn。

In [27]:
class RunnableMixIn(object):
    def run(self):
        print('Running...')
        
class FlyableMixIn(object):
    def fly(self):
        print('Flying...')

In [28]:
class Dog(Mammal, RunnableMixIn): 
    pass

MixIn的目的就是给一个类增加多个功能，这样在设计类的时候，优先考虑通过多重继承来组合多个MixIn的功能，而不是复杂的继承关系。

In [29]:
# MixIn案例
# Python自带了TCPServer和UDPServer这两类网络服务，而要同时服务多个用户就必须使用多进程或多线程模型，这两个模型有ForkingMixIn和ThreadingMixIn
# 提供。通过组合，我们就可以创造出合适的服务

In [30]:
# 编写一个多进程模式的TCP模型

# class MyTCPServer(TCPServer, ForkingMixIn):
#   pass

## 小结

由于Python允许使用多重继承，MixIn就是一种常见的设计。

只允许单一继承的语言(Java)不能使用MixIn的设计。

In [31]:
# 四、 定制类

# 看到类似__slots__这种形如__xxx__的变量或函数名就要注意，这些在Python中是有特殊用途。

In [32]:
## 4.1 __str__和__repr__

In [33]:
# 定义一个Student类，打印一个实例

class Student(object):
    def __init__(self, name):
        self.name = name

In [34]:
print(Student('A')) 

In [35]:
# <__main__.Student object at 0x000001F761CCEA88> 打印出这些也看不懂不好看
# 定义好__str__()方法，返回一个好看的字符串即可

class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name

In [36]:
print(Student('A'))

Student object (name: A)


In [37]:
s = Student('A')

In [38]:
s

In [39]:
# 以上打印出来并不好看，因为直接显示变量调用的不是__str__()，而是__repr__()，两者的区别前者返回用用户看到的字符串，而后者返回程序开发者看的
# 字符串，__repr__()是为了调式服务

In [40]:
# 解决办法再定义一个

class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name
    __repr__ = __str__

In [41]:
s = Student('B')

In [42]:
s

Student object (name: B)

In [43]:
## 4.2 __iter__
# 如果一个类想被用于for..in循环，类似list或tuple，就必须实现一个__iter__()方法，该方法返回一个迭代对象，然后Python的for循环不断调用该迭代
# 对象的__next__()方法，拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [44]:
# 斐波那契数列

class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b
    
    def __iter__(self):
        return self # 实例本身就是迭代对象，返回自己即可
    
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 100000: # 退出循环条件
            raise StopIteration()
        return self.a # 返回下一个值

In [45]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


In [46]:
## 4.3 __getitem__

In [47]:
# Fib实例虽然可以作用于for循环，看起来和list有点像，但是不能把它当作list使用

In [48]:
Fib()[5]

TypeError: 'Fib' object is not subscriptable

In [ ]:
# 像list那样按照下标取出元素，需要实现__getitem__()方法

In [49]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

In [50]:
f = Fib()

In [51]:
f[0]

1

In [52]:
f[1]

1

In [53]:
f[2]

2

In [54]:
# list有个神奇的切片方法

list(range(100))[5:10]

[5, 6, 7, 8, 9]

In [55]:
# 对于Fib报错。原因是__getitem__()传入的参数可能是一个int，也可能是一个切片对像slice，所以需要进行判断

In [56]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice):  # n是切片 
            start = n.start
            stop  = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            l =  []
            for x in range(stop):
                if x >= start:
                    l.append(a)
                a, b = b, a + b
            return l
                

In [57]:
f = Fib()

In [58]:
f[0:5]

[1, 1, 2, 3, 5]

In [59]:
# 没有对steop参数做处理

f[:10:2]

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [60]:
# 也没有对负数做处理，所以，要正确实现一个__getitem__()还是需要很多工作。
# 此外如果对象看成dict，__getitem__()的参数也可能是一个可以作为key的object。
# __setitem__()方法，把对象视作list或者dict来对集合赋值，__delitem__()方法用于删除某个元素。
# 我们定义的类表现得和Python自带的list、tuple、dict没什么区别，完全归功于动态语言的“鸭子类型”
# 不需要强制继承某个接口

In [61]:
## 4.4 __getattr__
# 正常情况下，当调用类的方法或属性时，如果不存在就报错。

In [62]:
class Student(object):
    
    def __init__(self):
        self.name = 'A'

In [63]:
# 调用name属性，不存在score属性

s = Student()

In [64]:
print(s.name)

A


In [65]:
print(s.score) # 报错：没有找到score这个attribute

AttributeError: 'Student' object has no attribute 'score'

In [ ]:
# 避免以上错误，除了加上score属性，还可以写一个__getattr__()方法，动态返回一个属性

class Student(object):
    
    def __init__(self):
        self.name = 'A'
        
    def __getattr__(self, attr):
        if attr=='score':
            return 99

In [ ]:
# 当调用不存在的属性时，比如score，Python解释器会试图调用__getattr__(self, 'score')来尝试获得属性

s = Student()

In [ ]:
s.name

In [ ]:
s.score

In [ ]:
# 返回函数也是可以的

class Student(object):
    
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25

In [ ]:
s = Student()

# 更改调用方式

s.age()

In [66]:
# 注意，只有在没有找到属性的情况下，才调用__getattr__，已有的属性，比如name，就不会在__getattr__中查找
# 注意到任何调用s.abc都会返回None,这是我们定义的__getattr__默认返回就是None。
# 要让class之相应特定的属性，就按照约定，抛出AttributeError的错误

In [67]:
class Stuedent(object):
    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)                    

这实际上可以把一个类的所有属性和方法调用全部动态化处理，不需要任何特殊手段。

这种完全动态调用的特性的作用就是可以针对完全动态的情况做调用。

In [68]:
# 利用完全动态的__getattr__,可以写出一个链式调用

class Chain(object):
    
    def __init__(self, path=''):
        self._path = path
        
    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))
    
    def __str__(self):
        return self._path
    
    __repr__ = __str__

In [69]:
Chain().status.user.timeline.list

/status/user/timeline/list

无论API怎么变，SDK都可以根据URL实现动态的调用，呃呃企鹅，不随API的增加而改变。

In [70]:
## 4.4 __call__

In [71]:
# 一个对象实例可以有自己的属性和方法，当我们调用实例方法时，我们用instacne.method()来调用。

In [72]:
# 任何类，只需要定义一个__call__()方法，就可以直接对实例进行调用

class Student(object):
    def __init__(self, name):
        self.name = name
        
    def __call__(self):
        print('My name is %s.' % self.name)

In [73]:
s = Student('A')

In [74]:
s() # self参数不要传入

My name is A.


In [75]:
# __call__()还可以定义参数。对实例进行直接调用就跟对一个函数进行调用一样。

In [76]:
# 判断一个对象是否能被调用，能被调用的对象就是一个callable对象

callable(max)

True

In [77]:
callable([1, 2, 3])

False

In [78]:
callable(None)

False

In [79]:
callable('str')

False

## 小结

Python的class允许定义许多定制方法，可以方便地生成特定的类。

In [80]:
# 五、 使用枚举类

In [81]:
# 需要定义常量时，一个办法是用大写变量通过整数来定义

In [82]:
JAN = 1
FEB = 2
MAR = 3

好处是简单，缺点是类型是int，并且仍然是变量。

In [83]:
# 这样的枚举类型定义一个class类型，然后每个常量都是class的一个唯一实例。Python提供了Enum的类来实现这个功能

In [84]:
from enum import Enum

In [85]:
Month = Enum('Month', ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'))

这样就获得了Month类型的枚举类，可以直接使用Month.Jan来引用一个常量，或者枚举它的所有成员

In [86]:
for  name, member in Month.__members__.items():
    print(name, '=>', member, ',', member.value)

Jan => Month.Jan , 1
Feb => Month.Feb , 2
Mar => Month.Mar , 3
Apr => Month.Apr , 4
May => Month.May , 5
Jun => Month.Jun , 6
Jul => Month.Jul , 7
Aug => Month.Aug , 8
Sep => Month.Sep , 9
Oct => Month.Oct , 10
Nov => Month.Nov , 11
Dec => Month.Dec , 12


value属性则是自动赋给成员的int常量，默认从1开始计数

In [87]:
# 需要更精确地控制枚举类型，可以从Enum派生出自定义类

from enum import Enum, unique

In [88]:
@unique
class Weekday(Enum):
    Sun = 0
    Mon = 1
    Tue = 2
    Wed = 3
    Thu = 4
    Fri = 5
    Sat = 6

In [89]:
# @unique装饰器可以帮助我们检查保证没有重复值

In [90]:
day1 = Weekday.Mon

In [91]:
day1

<Weekday.Mon: 1>

In [92]:
Weekday.Tue

<Weekday.Tue: 2>

In [93]:
Weekday['Tue']

<Weekday.Tue: 2>

In [94]:
Weekday.Tue.value

2

In [95]:
Weekday(1)

<Weekday.Mon: 1>

In [96]:
day1 == Weekday.Mon

True

In [97]:
Weekday(7)

ValueError: 7 is not a valid Weekday

In [ ]:
for  name, member in Weekday.__members__.items():
    print(name, '=>', member, ',', member.value)

可以用成员名称引用枚举常量，也可以直接根据value的值获得枚举常量

## 小结

Enum可以把一种相关常量定义在一个class中，且class不可变，而且成员可以直接比较

In [98]:
# 六、 使用元类

In [99]:
## 6.1 type()
# 动态语言和静态语言最大的不同，就是函数和类的定义，不是编译时定义，而是运行时动态创建

In [100]:
# 定义一个Hello的class，就写一个hello.py模块

In [101]:
class Hello(object):
    def hello(self, name='world'):
        print('hello, %s' % name)

In [102]:
# 当Python解释器载入hello模块，就会依次执行该模块的所有语句，执行结果就是动态创建出一个Hello的class对象

In [103]:
# >>> from hello import Hello
# >>> h = Hello()
# >>> h.hello
# Hello, world.
# >>> type(Hello)
# <class 'type'>
# >>> type(h)
# <class 'hello.Hello'>

type()函数可以查看一个类型或变量的类型，Hello是一个class，它的类型就是type，而h是一个实例，它的类型就是class Hello

In [104]:
# type()函数既可以返回一个对象的类型，还可以创建出新的类型

In [105]:
# >>> def fn(self, name='world'): # 定义函数
#      print('Hello, %s.' % name)
# >>> Hello = type('Hello', (object,), dict(hello=fn)) # 创建Hello class
# >>> h = Hello()
# >>> h.hello()
# Hello, world.
# >>> print(type(Hello))
# <class 'type'>
# >>> print(type(h))
# <class '__main__.Hello'>

要创建一个class对象，type()函数依次传入三个参数：

1.class的名称

2.继承的父类集合，注意Python支持多重继承，只有一个父类，别忘了tuple的单元素写法


3.class的方法名称与函数绑定，这里把fn绑定到hello上

In [106]:
## 6.2 metaclass（了解）

除了使用type()动态创建类以外，要控制类的创建行为，还可以使用metaclass。

metaclass，直译为元类，当定义了类以后，就可以根据这个类创建出实例，所以先定义类再创建实例。

如果想创建出类，就必须根据metaclass创建出类，先定义metaclass然后创建类。

总的来说，先定义metaclass，就可以创建类，最后创建实例。

In [107]:
# metaclass给我们自定义的MyList增加一个add方法
# 定义ListMetaclass，按照默认习惯，metaclass的类名总是以Metaclass结尾，以便清楚地表示这是一个metaclass

class ListMetaclass(type): # metaclass是类的模板，必须从type类型派生
    def __new__(cls, name, bases, attrs):
        attrs['add'] = lambda self, value: self.append(value)
        return type.__new__(cls, name, bases, attrs)

In [108]:
# 有了ListMetaclass，在定义类的时候还要指示使用ListMetaclass来定制类，传入关键字参数metaclass

class Mylist(list, metaclass=ListMetaclass):
    pass

In [109]:
# 传入关键字metaclass时，魔术生效，指示Python解释器再创建Mylist时，要通过ListMetaclass.__new__()来创建
# __new__()方法接收到的参数依次是
# 1.当前准备创建的类的对象
# 2.类的名字
# 3.类继承的父类集合
# 4.类的方法集合

In [110]:
# 测试一下Mylist是否可以调用add()方法

L = Mylist()

In [111]:
L.add(1)

In [112]:
L

[1]

In [113]:
# 普通的list没有add()方法

L2 = list()

In [114]:
L2.add(1)

AttributeError: 'list' object has no attribute 'add'

In [116]:
# ORM全称Object Relational Mapping，即对象-关系映射，就是把关系数据库的一行映射为一个对象，一个类对应一个表

# ORM框架
# 编写底层模块的第一步就是先调用接口写出来（定义一个User类操作对应的数据库表User）

#class User(Model):
#   # 定义类的属性到列的映射
#   id = IntegerField('id')
#    name = StringField('username')
#    email = StringField('email')
#    password = StringField('password')

# 创建一个实例
# u = User(id=123, name'A', email='123@qq.com', password='123')

# 保存到数据库
# u.save()

其中，父类Model和属性类型StringField、IntegerField是由ORM框架提供，剩下的魔术方法有metaclass自动完成。

In [117]:
# 实现ORM
# 定义Field类，负责保存数据表的字段名和字段类型

class Field(object):
    
    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type
        
    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

In [118]:
class StringField(Field):
    
    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

In [119]:
class IntegerField(Field):
    
    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

In [120]:
class ModelMetaclass(type):
    
    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表命和类名一样
        return type.__new__(cls, name, bases, attrs)

In [121]:
class Model(dict, metaclass=ModelMetaclass):
    
    def __init__(self, **kw):
        super(Model, self).__init__(**kw)
        
    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r'''Model' object has no attribute '%s''' % key)
    
    def __setattr__(self, key, value):
        self[key] = value
        
    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

当用户顶一个class User(Model)时，Python解释器首先在当前类User的定义中查找metaclass，如果没有找到，就继续在父类Model中查找metaclass，找到了就是用Model中定义的metaclass的ModelMetaclass来创建User类。

In [122]:
# ModelMetaclass做了 
# 1.排除掉对Model类的修改
# 2.在当前类中查找定义的类的所有类型
# 3.把表名存到__table__中，简化为表命默认为类名

In [123]:
class User(Model):
   # 定义类的属性到列的映射
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

Found model: User
Found mapping: id ==> <IntegerField:id>
Found mapping: name ==> <StringField:username>
Found mapping: email ==> <StringField:email>
Found mapping: password ==> <StringField:password>


In [124]:
u = User(id=123, name='A', email='123@qq.com', password='abc')

u.save()

SQL: insert into User (id,username,email,password) values (?,?,?,?)
ARGS: [123, 'A', '123@qq.com', 'abc']


## 小结

metaclass是Python中非常具有魔术性的对象，可以改变类创建时的行为。